In [1]:
import sys
import os

# 프로젝트 루트 디렉토리 경로를 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import jax
import jax.numpy as jnp
import time
#disable jax JIT
#jax.config.update("jax_disable_jit", True)

from tqdm.autonotebook import trange
from functools import partial
from JAxtar.bgpq import HashTableHeapValue, BGPQ

/tmp/ipykernel_25202/3628738232.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [3]:
heap = BGPQ.make_heap(100, 10, HashTableHeapValue)
insert = jax.jit(heap.insert)

In [4]:
heap_key = jax.random.uniform(jax.random.PRNGKey(0), shape=(3,), minval=0, maxval=10)
value = jax.vmap(HashTableHeapValue.default)(jnp.arange(3))
make_batch = BGPQ.make_batched(heap_key, value, 10)

In [5]:
for i in range(10):
    heap_key = jax.random.uniform(jax.random.PRNGKey(i), shape=(3,), minval=0, maxval=10)
    value = jax.vmap(HashTableHeapValue.default)(jnp.arange(3))
    block_key, block_val = BGPQ.make_batched(heap_key, value, 10)
    heap, buffer_oveflow = insert(heap, block_key[0], block_val[0])
    print(heap.key_store[0])
    print(heap.key_buffer)
    print(buffer_oveflow)

[3.1468165 6.3302994 9.653214        inf       inf       inf       inf
       inf       inf       inf]
[inf inf inf inf inf inf inf inf inf]
False
[2.586038  3.1468165 5.685525  6.3302994 8.893297  9.653214        inf
       inf       inf       inf]
[inf inf inf inf inf inf inf inf inf]
False
[2.586038  3.1468165 5.64795   5.685525  6.3302994 6.6586185 8.594385
 8.893297  9.653214        inf]
[inf inf inf inf inf inf inf inf inf]
False
[0.21006942 2.3853767  2.586038   3.1468165  5.64795    5.685525
 6.0624323  6.3302994  6.6586185  8.594385  ]
[8.893297 9.653214      inf      inf      inf      inf      inf      inf
      inf]
False
[0.21006942 2.3853767  2.586038   3.1468165  5.64795    5.685525
 6.0624323  6.0761747  6.3302994  6.6586185 ]
[7.2005796 8.594385  8.893297  9.653214  9.861052        inf       inf
       inf       inf]
False
[0.21006942 2.3853767  2.586038   2.630899   3.1468165  4.1553726
 5.64795    5.685525   6.0624323  6.0761747 ]
[6.3302994 6.6586185 7.2005796 8.3689